In [5]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

**Extract**

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter od dados do usuário correspondente.

In [6]:
import pandas as pd

df = pd.read_csv('SWD2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1771, 1774, 1775]


In [7]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None


users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1771,
    "name": "Thiaguinho",
    "account": {
      "id": 1859,
      "number": "00029-8",
      "agency": "string",
      "balance": 0.0,
      "limit": 550.0
    },
    "card": {
      "id": 1712,
      "number": "**** **** **** **** 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1774,
    "name": "Rhebequinha",
    "account": {
      "id": 1862,
      "number": "00049-8",
      "agency": "string",
      "balance": 0.0,
      "limit": 1550.0
    },
    "card": {
      "id": 1715,
      "number": "**** **** **** **** 2222",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1775,
    "name": "Luquinhas",
    "account": {
      "id": 1863,
      "number": "00045-8",
      "agency": "string",
      "balance": 0.0,
      "limit": 1550.0
    },
    "card": {
      "id": 1716,
      "number": "**** **** **** **** 3333",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]


**Transform**

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário

In [8]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 968.0 kB/s eta 0:00:00


In [9]:
openai_api_key = ""

In [11]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = [
          {
              "role": "system",
              "content": "Você é um especialista em marketing bancário."},
          {
              "role": "user",
              "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
          }
      ]
  )
  return completion.choices[0].message.content.strip('\"')


for user in users:
  news = generate_ai_news(user)
  print(news)




Invista agora e garanta seu futuro!
Invista hoje, colha seu futuro. #RhebequinhaInvista
Luquinhas, invista agora para garantir um futuro financeiramente estável!💰


**Load**

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.


In [12]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Thiaguinho updated? True
User Rhebequinha updated? True
User Luquinhas updated? True
